## Documentation: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html

Initilizing SQLContext

In [1]:
import pyspark
!rm -rf metastore_db/
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

To use RDD in SQLContext, RDD lines have to be converted into a Row format

In [2]:
import re
from pyspark.sql import Row # https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.Row
# Read from CSV
def load_csv(line):
    return re.split("\"?,\"?", line)
        
def readInt(x):
    try:
        return int(x)
    except:
        return x

def parseElement(e):
    return Row( year=readInt(e[0]),
                month=readInt(e[1]),
                day=readInt(e[2]),
                dep_time=readInt(e[3]),
                dep_delay=readInt(e[4]),
                arr_time=readInt(e[5]),
                arr_delay=readInt(e[6]),
                cancelled=e[7],
                carrier=e[8],
                tailnum=e[9],
                flight=readInt(e[10]),
                origin=e[11],
                dest=e[12],
                air_time=readInt(e[13]),
                distance=readInt(e[14]),
                hour=readInt(e[15]),
                min=readInt(e[16])
    )

In [3]:
!wget -P /tmp https://dsr-data.s3.amazonaws.com/flights/flights14.csv
#!hadoop fs -put /tmp/flights14.csv /tmp

--2017-01-25 15:30:22--  https://dsr-data.s3.amazonaws.com/flights/flights14.csv
Resolving dsr-data.s3.amazonaws.com (dsr-data.s3.amazonaws.com)... 52.219.73.22
Connecting to dsr-data.s3.amazonaws.com (dsr-data.s3.amazonaws.com)|52.219.73.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16150465 (15M) [text/csv]
Saving to: ‘/tmp/flights14.csv.2’

flights14.csv.2     100%[===================>]  15,40M  7,15MB/s    in 2,2s    

2017-01-25 15:30:25 (7,15 MB/s) - ‘/tmp/flights14.csv.2’ saved [16150465/16150465]



In [5]:
flights = sc.textFile("/tmp/flights14.csv").map(load_csv).filter(lambda e: not e[0] == "year").map(parseElement).cache()

Now the schema and the RDD have to be registered with the sqlContext:

In [6]:
flightsDF = sqlContext.createDataFrame(flights) #create a dataframe

In [7]:
flightsDF.registerTempTable("flight") #register as a table

In [8]:
sqlContext.sql("select * from flight where dest = 'LAX'").take(5)

[Row(air_time=359, arr_delay=13, arr_time=1238, cancelled='0', carrier='AA', day=1, dep_delay=14, dep_time=914, dest='LAX', distance=2475, flight=1, hour=9, min=14, month=1, origin='JFK', tailnum='N338AA', year=2014),
 Row(air_time=363, arr_delay=13, arr_time=1523, cancelled='0', carrier='AA', day=1, dep_delay=-3, dep_time=1157, dest='LAX', distance=2475, flight=3, hour=11, min=57, month=1, origin='JFK', tailnum='N335AA', year=2014),
 Row(air_time=351, arr_delay=9, arr_time=2224, cancelled='0', carrier='AA', day=1, dep_delay=2, dep_time=1902, dest='LAX', distance=2475, flight=21, hour=19, min=2, month=1, origin='JFK', tailnum='N327AA', year=2014),
 Row(air_time=350, arr_delay=1, arr_time=1706, cancelled='0', carrier='AA', day=1, dep_delay=2, dep_time=1347, dest='LAX', distance=2475, flight=117, hour=13, min=47, month=1, origin='JFK', tailnum='N319AA', year=2014),
 Row(air_time=339, arr_delay=0, arr_time=2145, cancelled='0', carrier='AA', day=1, dep_delay=4, dep_time=1824, dest='LAX', d

In [9]:
flightsDF.where("origin = 'JFK' AND dest = 'MIA'").count()

2750

In [10]:
flightsDF.where("origin = 'JFK' AND dest = 'MIA'").limit(2).collect()

[Row(air_time=161, arr_delay=-17, arr_time=1828, cancelled='0', carrier='AA', day=1, dep_delay=-1, dep_time=1509, dest='MIA', distance=1089, flight=145, hour=15, min=9, month=1, origin='JFK', tailnum='N5FJAA', year=2014),
 Row(air_time=166, arr_delay=-8, arr_time=1227, cancelled='0', carrier='AA', day=1, dep_delay=7, dep_time=917, dest='MIA', distance=1089, flight=1085, hour=9, min=17, month=1, origin='JFK', tailnum='N5DWAA', year=2014)]

### Count the flights that departed early and arrived late

In [15]:
flightsDF.where("dep_delay < 0 AND arr_delay > 0").count()

30239

In [21]:
flightsDF.describe().show()

+-------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------+
|summary|          air_time|        arr_delay|          arr_time|               day|         dep_delay|          dep_time|          distance|            flight|              hour|               min|            month|  year|
+-------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------+
|  count|            253316|           253316|            253316|            253316|            253316|            253316|            253316|            253316|            253316|            253316|           253316|253316|
|   mean|156.72282445641017| 8.14670214277819|1494.3299002036981|15.893697989862464|12.465264728639328|1

In [20]:
flightsDF.columns  ## list of columns

['air_time',
 'arr_delay',
 'arr_time',
 'cancelled',
 'carrier',
 'day',
 'dep_delay',
 'dep_time',
 'dest',
 'distance',
 'flight',
 'hour',
 'min',
 'month',
 'origin',
 'tailnum',
 'year']

In [19]:
flightsDF.printSchema()     # To see the datatype of columns

root
 |-- air_time: long (nullable = true)
 |-- arr_delay: long (nullable = true)
 |-- arr_time: long (nullable = true)
 |-- cancelled: string (nullable = true)
 |-- carrier: string (nullable = true)
 |-- day: long (nullable = true)
 |-- dep_delay: long (nullable = true)
 |-- dep_time: long (nullable = true)
 |-- dest: string (nullable = true)
 |-- distance: long (nullable = true)
 |-- flight: long (nullable = true)
 |-- hour: long (nullable = true)
 |-- min: long (nullable = true)
 |-- month: long (nullable = true)
 |-- origin: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- year: long (nullable = true)



### Find the flightwith the longest arrival delay

In [22]:
flightsDF.orderBy(flightsDF.arr_delay.desc()).first()

Row(air_time=200, arr_delay=1494, arr_time=1008, cancelled='0', carrier='AA', day=4, dep_delay=1498, dep_time=727, dest='DFW', distance=1372, flight=1381, hour=7, min=27, month=10, origin='EWR', tailnum='N4WJAA', year=2014)

### Find the top 10 destinations ordered by the number of flights

In [30]:
group_dest_cts = flightsDF.groupBy(flightsDF.dest).count()  ### Groups by destinations and counts
group_dest_cts.printSchema() 

root
 |-- dest: string (nullable = true)
 |-- count: long (nullable = false)



In [45]:
group_dest_cts.show(5)

+----+-----+
|dest|count|
+----+-----+
| PSE|  289|
| MSY| 2628|
| BUR|  289|
| SNA|  657|
| GRR|  756|
+----+-----+
only showing top 5 rows



In [57]:
group_dest_cts.sort("count", ascending = False).show(10)

+----+-----+
|dest|count|
+----+-----+
| LAX|14434|
| ATL|12808|
| SFO|11907|
| MCO|11709|
| BOS|11609|
| ORD|11589|
| MIA| 9928|
| CLT| 9624|
| FLL| 9471|
| DCA| 6748|
+----+-----+
only showing top 10 rows



### Find top 10 destinations with the worst avg arrival delay, ignoring flights that arrived early

In [72]:
delayed_flights = flightsDF.where("arr_delay > 0")
delayed_flights_sum = delayed_flights.groupby(delayed_flights.dest).agg({'arr_delay': 'mean'})
delayed_flights_sum.sort('avg(arr_delay)', ascending = False).show(10)

+----+------------------+
|dest|    avg(arr_delay)|
+----+------------------+
| EGE| 80.59016393442623|
| AVP|              67.0|
| JAC|59.666666666666664|
| CAK|57.907042253521126|
| TUL| 56.99152542372882|
| MSN| 54.80237154150198|
| IAD|53.324599708879184|
| BGR| 52.73504273504273|
| OKC| 52.14503816793893|
| TVC| 51.15151515151515|
+----+------------------+
only showing top 10 rows



### Take a sample of 1% of the flights and then calculate the average departure delay for that sample

### For all flights from JFK during June, show the average departure delay for each destination

### For every origin/dest pair, count the number of flights